In [ ]:
os.chdir('..')

In [ ]:

from model_config import *
from model_packages import *
from model_utils import *
import re

# Load Model

In [ ]:

model = load_model(data_folder+'voting_regressor_model') 

# Format input data

In [ ]:
ts_input_data= pd.read_pickle(data_folder+f'test_sites_static_and_dynamic_features_{bufr_zones_mrts}.pkl')
ts_input_data[['amenity_bar', 'amenity_bus_station', 'amenity_food_court', 'amenity_taxi_station', 'amenity_coach_parking', 'amenity_beer_garden']] = 0

df, lnd_ftrs, hbt_ftrs= preprocess_input_data(ts_input_data)

# create dummy variables for categorical data
df=pd.get_dummies(df,columns=['land_type_labels'])
df=pd.get_dummies(df,columns=['land_habitat_labels'])
df['counter']= df['site']

In [ ]:
# If test locations dont have certain features present e.g. no 'amenity_pub' 
# or no test location labled 'land_type_labels_rural_settings' these columns must be added 


df[['land_habitat_labels_Grassland_woodland_bareground', 'land_type_labels_rural_settings', 'land_type_labels_urban_settings']]= 0
df[['land_habitat_labels_Grassland_woodland_bareground', 'land_type_labels_rural_settings', 'land_type_labels_urban_settings']]= df[['land_habitat_labels_Grassland_woodland_bareground', 'land_type_labels_rural_settings', 'land_type_labels_urban_settings']].astype(np.uint8)

# Generate Predictions

In [ ]:
# columns of input data set must match the columns present when training the model
# these can be found in data/training_predictions.pkl
test_site_predictions= predict_model(model, data=df[['counter','Date', 'Dog', 'accessible_green_space_area', 'PROW_Total_length_km', 'waterside_length_km', 'land_type_labels_mixed_settings', 'land_habitat_labels_Grassland_woodland_wetland', 'amenity_bar', 'amenity_parking', 'amenity_pub', 'amenity_toilets', 'amenity_beer_garden', 'amenity_bus_station', 'amenity_food_court', 'amenity_taxi_station', 'amenity_coach_parking', 'Age group 0-25', 'Age group 25-65', 'Household is deprived in at least 1 dimension', 'Unemployed_population', 'Asian, Asian British or Asian Welsh', '2 or more cars or vans in household', 'Population Density: Persons per square kilometre; measures: Value', 'total_trip_count', 'tavg']])